<h1>Desafio 1 - Grupo 5</h1>

<h3><u>Intregrantes del grupo</u></h3>

- Lesertesseur, Diego
- Panizza, Camila  
- Pellecchia, Franco   
- Magariños, Néstor  
- Suarez, Horacio  

<h3><u>Objetivos</u></h3>
La inmobiliaria Properati publica periódicamente información sobre ofertas de propiedades
para venta y alquiler. Ud. deberá asesorar a la inmobiliaria a desarrollar un modelo de
regresión que permita predecir el precio por metro cuadrado de una propiedad. El objetivo
final es que el modelo que desarrollen sea utilizado como tasador automático a ser
aplicados a las próximas propiedades que sean comercializadas por la empresa. Para ello
la empresa le provee de un dataset correspondiente al primer semestre de 2017.

- Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.
- Realizar un análisis descriptivo de las principales variables.
- Crear nuevas columnas a partir de las características dadas que puedan tener valor
predictivo.

<h3><u>Variables</u></h3>

- Fecha de creación
- Tipo de la propiedad (house, apartment, ph)
- Operación del aviso (sell, rent)
- Nombre del lugar
- Nombre del lugar + nombre de sus ‘padres’
- ID de geonames del lugar (si está disponible)
- Latitud,Longitud
- Precio original del aviso
- Moneda original del aviso (ARS, USD)
- Precio del aviso en moneda local (ARS)
- Precio aproximado en USD
- Superficie en m²
- Superficie cubierta en m²
- Precio en USD/m²
- Precio por m²
- N° de piso, si corresponde
- Ambientes
- URL en Properati
- Descripción
- Título
- URL de un thumbnail de la

<h3><u>Índice</u></h3>
<a href='#section_1'>1. Preparación del dataset</a> 

<a href='#section_2'>2. Data wrangling</a>  
<a href='#section_3'>3. Nuevas variables</a>  
<a href='#section_4'>4. Dataset final</a>

---
---
<b><a id='section_1'><h4>1. Preparación del dataset</h4></a></b>
<a href='#section_1_1'>1.1. Importación de librerías y carga del dataset</a>  
<a href='#section_1_2'>1.2. Descripción del dataset</a>  
<a href='#section_1_3'>1.3. Selección de columnas útiles</a>  
<a href='#section_1_4'>1.4. Detección de duplicados</a>

<b><a id='section_1_1'>1.1. Importación de librerías y carga del dataset</a>  </b>

In [ ]:
import numpy as np
import pandas as pd
import re 
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import math

In [ ]:
data= pd.read_csv("C:/Users/fmpel/Downloads/properati (1).csv")

---
<b><a id='section_1_2'>1.2. Descripción del dataset</a></b>

In [ ]:
data_describe = pd.DataFrame(round(data.isnull().sum() /data.shape[0],2), columns= ['Porcentaje de nulos'])
data_describe['type'] = data.dtypes
data_describe = data_describe[['type','Porcentaje de nulos']]
data_describe['Nulos'] = data.isnull().sum()
values= []
example=[]
for x in data:
    values.append(len(data[x].unique()))
    example.append(data[x].unique())
data_describe['Valores distintos'] = values
data_describe['Ejemplos'] = example

data_describe

---
<b><a id='section_1_3'>1.3. Selección de columnas útiles</a></b>

In [ ]:
columnasUtiles=   ['property_type','place_name','state_name', 'place_with_parent_names', 'price', 'currency','price_aprox_local_currency',
                   'price_aprox_usd','surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2',
                   'rooms', 'expenses','description','title']
data_describe_utiles = data_describe.loc[columnasUtiles,:]
data_describe_utiles

---
<b><a id='section_1_4'>1.4. Deteción de duplicados</a></b>  
Se procede a eliminar las observaciones duplicadas según las variables indicadas en 'columnasDuplicadas'. Se opta por conservar las observaciones que tienen descripción más larga

In [ ]:
columnasDuplicadas = ['property_type','place_name','state_name', 'place_with_parent_names',
                      'surface_total_in_m2','surface_covered_in_m2','title']
data[columnasDuplicadas].duplicated().sum()

In [ ]:
data['description_len'] = data['description'].apply(lambda x: len(str(x)))
data = data.sort_values(by=['description_len'], ascending= False)

In [ ]:
display(data.shape)
data = data.drop_duplicates(subset = columnasDuplicadas, keep= 'first')
display(data.shape)

In [ ]:
data = data.reset_index()

---
---

<b><a id='section_2'><h4>2. Data wrangling</h4></a></b>
<a href='#section_2_1'>2.1. Columna 'property_type'</a>  
<a href='#section_2_2'>2.2. Columnas 'country_name', 'state_name', 'place_name' y'place_with_parent_names'</a>  
<a href='#section_2_3'>2.3. Columnas 'price','currency','price_aprox_local_currency' y 'price_aprox_usd' </a>  
<a href='#section_2_4'>2.4. Columna 'rooms'</a>  
<a href='#section_2_5'>2.5. Columna 'floor'</a>  
<a href='#section_2_6'>2.6. Columnas 'surface_total_in_m2' y 'surface_covered_in_m2'</a>  
<a href='#section_2_7'>2.7. Columna 'price_usd_per_m2'</a>  

In [ ]:
data['description_lower'] = data['description'].str.lower()
data['title_lower'] = data['title'].str.lower()

<b><a id='section_2_1'>2.1. Columna 'property_type'</a></b>

Análisis de valores y nulos

In [ ]:
data_describe.loc[data_describe.index.isin(['property_type']),:]

Visualización

In [ ]:
f, ax = plt.subplots()
f.set_figwidth(7)
f.set_figheight(7)
sns.barplot(y=data['property_type'].value_counts(),x=data['property_type'].value_counts().index)
plt.title("Distribución de propiedades según tipo",fontsize=15)
plt.legend(labels=data['property_type'].value_counts().index)
pd.DataFrame(round((data['property_type'].value_counts()/data.shape[0])*100,1))

---
<b><a id='section_2_2'>2.2. Columnas 'country_name', 'state_name', 'place_name' y'place_with_parent_names'</a></b>

Análisis de valores y nulos

In [ ]:
data_describe.loc[data_describe.index.isin(['country_name', 'state_name', 'place_name','place_with_parent_names']),:]

Dividimos la columna 'place_with_parent_names' y analizamos su contenido.

In [ ]:
data['place_with_parent_names_clean'] = data['place_with_parent_names']

total_place = data['place_with_parent_names_clean'].apply(lambda row: len(row.split('|'))-2)
dic_ret = dict()
for i in range(total_place.max()):
    dic_ret['place_'+ str(i)] = ([None] * total_place.shape[0])

In [ ]:
for row in data['place_with_parent_names_clean'].index:
    i=0;
    list_places = data['place_with_parent_names_clean'].loc[row].split('|')
    for x in list_places:
        if len(x)>0:
            colname='place_' + str(i)
            dic_ret[colname][row] = x
            i+=1

In [ ]:
new_columns = pd.DataFrame(dic_ret)
data = data.join(new_columns)

In [ ]:
data[['country_name','state_name','place_name','place_0','place_1','place_2','place_3','place_4']].sample(10)

La columna place_with_parent_names contiene la misma información que las columnas 'country_name', 'state_name' y 'place_name' e incluye información adicional.  
Se opta por utilizar la información proveniente de 'place_with_parent_name' y <u>se generan las variables 'provincia','ciudad','departamento' y'barrio'</u>

In [ ]:
data[['provincia','ciudad','departamento','barrio']] = data[['place_1','place_2','place_3','place_4']]

Visualización

In [ ]:
patternBsAs = re.compile("Buenos Aires\s*")
data['provincia'] = data['provincia'].apply(lambda x: patternBsAs.sub('Bs.As. ', x))

f, ax = plt.subplots()
f.set_figwidth(15)
f.set_figheight(12)
sns.barplot((data['provincia'].value_counts()/data['provincia'].shape[0])*100,data['provincia'].unique(), ax=ax , color = "#009966", orient="h")
plt.title("Distribución de propiedades según provincia\n\n",fontsize=15)
plt.xlabel("Porcentaje",fontsize=11)
ax.xaxis.set_label_position('top') 
plt.ylabel("Provincia",fontsize=11)
ax.xaxis.tick_top()

---
<b><a id='section_2_3'>2.3. Columnas 'price','currency','price_aprox_local_currency' y 'price_aprox_usd' </a>  </b>

Análisis de valores y nulos

In [ ]:
data_describe.loc[data_describe.index.isin(['price','currency','price_aprox_local_currency','price_aprox_usd']),:]

In [ ]:
maskNull_priceAndCurrency = data['price'].isnull() & data['currency'].isnull() &\
                            data['price_aprox_local_currency'].isnull() &\
                            data['price_aprox_usd'].isnull()

print("Cantidad de nulos que coinciden:",maskNull_priceAndCurrency.value_counts()[True],"de 20411")

In [ ]:
data.loc[data['currency'].isnull() & data['price'].notnull() ,['price','currency','price_aprox_local_currency','price_aprox_usd']]

Detectamos que <u>los nulos de las columnas  'price','currency','price_aprox_local_currency' y 'price_aprox_usd' coinciden</u>  
Existe un solo nulo adicional en la columna 'currency' en el cual el resto toma valor 0. Se decide reemplazar esos 0 por NaN

In [ ]:
data[['price_clean', 'price_aprox_local_currency_clean','price_aprox_usd_clean']]=data[['price','price_aprox_local_currency','price_aprox_usd']]

In [ ]:
data.loc[data['currency'].isnull(),['price_clean', 'price_aprox_local_currency_clean','price_aprox_usd_clean']] = np.NaN
data.loc[9761,['price_clean', 'currency', 'price_aprox_local_currency_clean','price_aprox_usd_clean']]

Para la propiedades que tienen 'price' expresado en USD, imputamos estos valores en la columna 'price_aprox_usd_clean'

In [ ]:
maskUSD = data['currency'] == "USD"
data.loc[maskUSD, 'price_aprox_usd_clean'] = data.loc[maskUSD, 'price']

Visualización

In [ ]:
maskNotNull_price= data['price_aprox_usd_clean'].notnull()
precioMaximo=data['price_aprox_usd_clean'].max()
bins= [0,75000,150000,225000,300000,375000,450000, 525000, 600000, precioMaximo]

dist = pd.cut(data.loc[maskNotNull_price, 'price_aprox_usd_clean'], bins) 

f, ax = plt.subplots()
f.set_figwidth(20)
f.set_figheight(4.5)
sns.barplot(dist.unique(),(dist.value_counts()/data.loc[maskNotNull_price, 'price_aprox_usd_clean'].shape[0])*100,ax=ax , color = "#009966")
plt.title("Distribución de propiedades según precio",fontsize=15)
plt.xlabel("Precio",fontsize=11)
plt.ylabel("Porcentaje",fontsize=11)

Análisis de outliers.   
  
Debido a que pueden existir significativas diferencias de valor entre las propiedades según el tipo de propiedad y la ciudad donde se ubican, se calculan valores límites para determinar outliers a partir de estas variables. Esta consideración es tenida en cuenta en todos los procesos de "Análisis de outliers"

In [ ]:
data['outliers_Price'] = None
data['ciudad_sinNulos'] = data['ciudad'].fillna("-")
data['auxiliar_Outliers'] = data['property_type']+data['provincia']+data['ciudad_sinNulos']
for i in data['auxiliar_Outliers'].unique():
    rangoInterquartil_price= data.loc[data['auxiliar_Outliers']==i,'price_aprox_usd_clean'].quantile(0.75) - data.loc[data['auxiliar_Outliers']==i,'price_aprox_usd_clean'].quantile(0.25)
    umbral_lower_Price = data.loc[data['auxiliar_Outliers']==i,'price_aprox_usd_clean'].quantile(0.25) - rangoInterquartil_price*1.5
    umbral_upper_Price = data.loc[data['auxiliar_Outliers']==i,'price_aprox_usd_clean'].quantile(0.75) + rangoInterquartil_price*1.5


    data.loc[maskNotNull_price & (data['auxiliar_Outliers']==i), 'outliers_Price']= data.loc[maskNotNull_price & (data['auxiliar_Outliers']==i), 'price_aprox_usd_clean'].apply\
                                                                                    (lambda x: "Outlier superior" if x >umbral_upper_Price else(
                                                                                 "Outlier inferior" if x<umbral_lower_Price else 
                                                                                 "No es outlier"))

outliers_Price = data['outliers_Price']
data = data.drop(columns = ['outliers_Price','ciudad_sinNulos'])
pd.DataFrame(round((outliers_Price.value_counts() / outliers_Price[maskNotNull_price].shape[0])*100,1))

Se reemplaza a los outliers por nulos

In [ ]:
maskOutliers= outliers_Price !="No es outlier"
data.loc[maskOutliers,'price_aprox_usd_clean'] = np.NaN

Visualización sin outliers

In [ ]:
maskNotNull_price= data['price_aprox_usd_clean'].notnull()
precioMaximo=data['price_aprox_usd_clean'].max()
bins= [0,75000,150000,225000,300000,375000,450000, precioMaximo]

dist = pd.cut(data.loc[maskNotNull_price, 'price_aprox_usd_clean'], bins) 

f, ax = plt.subplots()
f.set_figwidth(20)
f.set_figheight(4.5)
sns.barplot(dist.unique(),(dist.value_counts()/data.loc[maskNotNull_price, 'price_aprox_usd_clean'].shape[0])*100,ax=ax , color = "#009966")
plt.title("Distribución de propiedades según precio (sin outliers)",fontsize=15)
plt.xlabel("Precio",fontsize=11)
plt.ylabel("Porcentaje",fontsize=11)

In [ ]:
maskNotNull_Rooms = data['price_aprox_usd_clean'].notnull()
f, ax = plt.subplots()
f.set_figwidth(12)
f.set_figheight(7)
sns.boxplot(x=data.loc[maskNotNull_Rooms,'property_type'],y=data.loc[maskNotNull_Rooms,'price_aprox_usd_clean'])
plt.title("Precio según tipo de propiedad",fontsize=15)
plt.xlabel("Tipo de propiedad",fontsize=11)
plt.ylabel("Precio",fontsize=11)

---
<b><a id='section_2_4'>2.4. Columna 'rooms'</a>  </b>

In [ ]:
data['rooms_clean'] = data['rooms']

Análisis de nulos y valores

In [ ]:
data_describe.loc[data_describe.index.isin(['rooms']),:]

Completamos los nulos de la columna 'rooms' con datos extraidos de la columna 'descripcion' y 'title'

In [ ]:
#Primer pattern
regexAmbientes = re.compile('(?P<numero>\d{1})\s?(amb)')

#Regex en la columna 'descripcion'
MatchsAmbientes = data['description_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes.search(x))
maskRooms_NullsAndMatch = data['rooms'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(lambda x: float(x.group("numero")))
#Regex en la columna 'title'
MatchsAmbientes = data['title_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes.search(x))
maskRooms_NullsAndMatch = data['rooms_clean'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(lambda x: float(x.group("numero")))

In [ ]:
#Segundo pattern
regexAmbientes2 = re.compile('(?P<numeroPalabra>uno|un|dos|tres|cuatro|cinco|seis|siete|ocho)\s?(amb)')

#Definimos una función para transformar el match obtenido en el search en un número
def MatchToNumber(x):
    numeroStr = x.group("numeroPalabra")
    dictAmbientes = {'un': 1, 'uno':1,'dos':2, 'tres':3, 'cuatro':4, 'cinco': 5, 'seis': 6, 'seís':6, 'siete': 7, 'ocho': 8, 'nueve': 9}
    resultado = dictAmbientes[numeroStr]
    return resultado

#Regex en la columna 'descripcion'
MatchsAmbientes = data['description_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes2.search(x))
maskRooms_NullsAndMatch = data['rooms_clean'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(MatchToNumber)

#Regex en la columna 'title'
MatchsAmbientes = data['title_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes2.search(x))
maskRooms_NullsAndMatch = data['rooms_clean'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(MatchToNumber)

In [ ]:
#Tercer pattern
regexAmbientes3 = re.compile('(?P<numero>\d{1})\s?(dorm|cuarto|hab)')
#Debido a que este Pattern considera únicamente dormitorios/habitaciones, se le agrega 1 habitación a todos los matchs

#Regex en la columna 'descripcion'
MatchsAmbientes = data['description_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes3.search(x))
maskRooms_NullsAndMatch = data['rooms'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(lambda x: float(x.group("numero"))+1)
#Regex en la columna 'title'
MatchsAmbientes = data['title_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes3.search(x))
maskRooms_NullsAndMatch = data['rooms_clean'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(lambda x: float(x.group("numero"))+1)


In [ ]:
#Cuarto pattern
regexAmbientes4 = re.compile('(?P<numeroPalabra>un|1|mono)\s?(amb)')

#Regex en la columna 'descripcion'
MatchsAmbientes = data['description_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes4.search(x))
maskRooms_NullsAndMatch = data['rooms_clean'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(lambda x: float(1.0))

#Regex en la columna 'title'
MatchsAmbientes = data['title_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes4.search(x))
maskRooms_NullsAndMatch = data['rooms_clean'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(lambda x: float(1.0))

In [ ]:
#Quinto pattern
regexAmbientes5 = re.compile('(?P<numeroPalabra>un|dos|tres|cuatro|cinco|seis|siete|ocho)\s?(dorm|cuarto|hab)')
#Debido a que este Pattern considera únicamente dormitorios/habitaciones, se le agrega 1 habitación a todos los matchs

#Regex en la columna 'descripcion'
MatchsAmbientes = data['description_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes5.search(x))
maskRooms_NullsAndMatch = data['rooms_clean'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(lambda x: MatchToNumber(x)+1)

#Regex en la columna 'title'
MatchsAmbientes = data['title_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes5.search(x))
maskRooms_NullsAndMatch = data['rooms_clean'].isnull() & MatchsAmbientes.notnull()
data.loc[maskRooms_NullsAndMatch, 'rooms_clean'] = MatchsAmbientes[maskRooms_NullsAndMatch].apply(lambda x: MatchToNumber(x)+1)

In [ ]:
print("Columna 'rooms'")
print("Cantidad de nulos antes del regex:", data['rooms'].isnull().sum(), "("+str(round((data['rooms'].isnull().sum()/data.shape[0])*100,1))+"%)")
print("Cantidad de nulos después del regex:", data['rooms_clean'].isnull().sum(),"("+str(round((data['rooms_clean'].isnull().sum()/data.shape[0])*100,1))+"%)")

Visualización

In [ ]:
maskNotNull_Rooms = data['rooms_clean'].notnull()
f, ax = plt.subplots()
f.set_figwidth(12)
f.set_figheight(7)
sns.boxplot(x=data.loc[maskNotNull_Rooms,'property_type'],y=data.loc[maskNotNull_Rooms,'rooms_clean'])
plt.title("Distribución de cuartos según propiedad",fontsize=15)
plt.xlabel("Tipo de propiedad",fontsize=11)
plt.ylabel("Cuartos",fontsize=11)

Análisis de outliers

In [ ]:
data['outliers_rooms'] = None
for i in data['property_type'].unique():
    rangoInterquartil_rooms= data.loc[data['property_type']==i,'rooms_clean'].quantile(0.75) - data.loc[data['property_type']==i,'rooms_clean'].quantile(0.25)
    umbral_lower_rooms = data.loc[data['property_type']==i,'rooms_clean'].quantile(0.25) - rangoInterquartil_rooms*1.5
    umbral_upper_rooms = data.loc[data['property_type']==i,'rooms_clean'].quantile(0.75) + rangoInterquartil_rooms*1.5


    data.loc[maskNotNull_Rooms & (data['property_type']==i), 'outliers_rooms']= data.loc[maskNotNull_Rooms & (data['property_type']==i), 'rooms_clean'].apply\
                                                                                    (lambda x: "Outlier superior" if x >umbral_upper_rooms else(
                                                                                 "Outlier inferior" if x<umbral_lower_rooms else 
                                                                                 "No es outlier"))

outliers_rooms = data['outliers_rooms']
data = data.drop(columns = ['outliers_rooms'])
pd.DataFrame(round((outliers_rooms.value_counts() / outliers_rooms[maskNotNull_Rooms].shape[0])*100,1))

Se reemplaza a los outliers por nulos

In [ ]:
maskOutliers= outliers_rooms !="No es outlier"
data.loc[maskOutliers,'rooms_clean'] = np.NaN

Le imputamos a los datos que los nulos el valor promedio de cuartos según su tipo de propiedad.
Los nulos proveniente de los outliers quedan nulos

In [ ]:
maskOldNulls = maskNotNull_Rooms ==False 

In [ ]:
meanByProperty_rooms = pd.DataFrame(data.groupby('property_type')['rooms_clean'].mean())
meanByProperty_rooms.columns = ['rooms_clean_mean']
meanByProperty_rooms['rooms_clean_mean'] = meanByProperty_rooms['rooms_clean_mean'].apply(lambda x: math.floor(x))
meanByProperty_rooms.sort_values(by = 'rooms_clean_mean', ascending=False)

In [ ]:
data= data.merge(meanByProperty_rooms, on='property_type')

In [ ]:
data.loc[maskOldNulls,'rooms_clean'] = data.loc[maskOldNulls,'rooms_clean_mean'] 
data = data.drop(columns=['rooms_clean_mean'])

Visualización sin outliers

In [ ]:
f, ax = plt.subplots()
f.set_figwidth(12)
f.set_figheight(7)
sns.boxplot(x=data['property_type'],y=data['rooms_clean'])
plt.title("Distribución de cuartos según propiedad (sin outliers)",fontsize=15)
plt.xlabel("Tipo de propiedad",fontsize=11)
plt.ylabel("Cuartos",fontsize=11)

---
<b><a id='section_2_5'>2.5. Columna 'floor'</a> </b>

In [ ]:
data['floor_clean'] = data['floor']

Análisis de nulos y valores

In [ ]:
data_describe.loc[data_describe.index.isin(['floor']),:]

In [ ]:
print("Campos con información en la columna 'floor' según tipo de propiedad")
pd.DataFrame(data.groupby('property_type')['floor'].apply(lambda x: x.notnull().sum()))

Se decide contemplar la variable 'floor' únicamente para los departamentos. Se le imputa valor nulo a los otros tipo de propiedad

In [ ]:
maskNotApartment = data['property_type'] != 'apartment'
data.loc[maskNotApartment, 'floor_clean'] = np.NaN
pd.DataFrame(data.groupby('property_type')['floor_clean'].apply(lambda x: x.notnull().sum()))

Completamos los nulos de la columna 'floor_clean' con datos extraidos de la columna 'descripcion' y 'title'

In [ ]:
maskApartment = data['property_type'] == 'apartment'

In [ ]:
#Primer pattern
regexFloor = re.compile('(?P<numeroPiso>\d\d?)\s?(^er|er|do|to|mo|ro|avo|vo|no)')

#Regex en la columna 'descripcion'
MatchsFloor = data['description_lower'].apply(lambda x: x if x is np.NaN else regexFloor.search(x))
maskFloor_NullsAndMatch = data['floor_clean'].isnull() & MatchsFloor.notnull()
data.loc[maskFloor_NullsAndMatch, 'floor_clean'] = MatchsFloor[maskFloor_NullsAndMatch].apply(lambda x: float(x.group("numeroPiso")))

#Regex en la columna 'title'
MatchsFloor = data['title_lower'].apply(lambda x: x if x is np.NaN else regexFloor.search(x))
maskFloor_NullsAndMatch = data['floor_clean'].isnull() & MatchsFloor.notnull()
data.loc[maskFloor_NullsAndMatch, 'floor_clean'] = MatchsFloor[maskFloor_NullsAndMatch].apply(lambda x: float(x.group("numeroPiso")))

In [ ]:
#Segundo pattern
regexFloor2 = re.compile('(?P<numeroPiso>\d\d?)\s?\º?\s?(piso)')

#Regex en la columna 'descripcion'
MatchsFloor = data['description_lower'].apply(lambda x: x if x is np.NaN else regexFloor2.search(x))
maskFloor_NullsAndMatch = data['floor_clean'].isnull() & MatchsFloor.notnull()
data.loc[maskFloor_NullsAndMatch, 'floor_clean'] = MatchsFloor[maskFloor_NullsAndMatch].apply(lambda x: float(x.group("numeroPiso")))

#Regex en la columna 'title'
MatchsFloor = data['title_lower'].apply(lambda x: x if x is np.NaN else regexFloor2.search(x))
maskFloor_NullsAndMatch = data['floor_clean'].isnull() & MatchsFloor.notnull()
data.loc[maskFloor_NullsAndMatch, 'floor_clean'] = MatchsFloor[maskFloor_NullsAndMatch].apply(lambda x: float(x.group("numeroPiso")))

In [ ]:
#Tercer pattern
regexFloor3 = re.compile('(piso)\s?(?P<numeroPiso>\d\d?)')

#Regex en la columna 'descripcion'
MatchsFloor = data['description_lower'].apply(lambda x: x if x is np.NaN else regexFloor3.search(x))
maskFloor_NullsAndMatch = data['floor_clean'].isnull() & MatchsFloor.notnull()
data.loc[maskFloor_NullsAndMatch, 'floor_clean'] = MatchsFloor[maskFloor_NullsAndMatch].apply(lambda x: float(x.group("numeroPiso")))

#Regex en la columna 'title'
MatchsFloor = data['title_lower'].apply(lambda x: x if x is np.NaN else regexFloor3.search(x))
maskFloor_NullsAndMatch = data['floor_clean'].isnull() & MatchsFloor.notnull()
data.loc[maskFloor_NullsAndMatch, 'floor_clean'] = MatchsFloor[maskFloor_NullsAndMatch].apply(lambda x: float(x.group("numeroPiso")))

In [ ]:
#Cuarto pattern
regexFloor3 = re.compile('pb|plantabaja|planta baja')

#Regex en la columna 'descripcion'
MatchsFloor = data['description_lower'].apply(lambda x: x if x is np.NaN else regexFloor3.search(x))
maskFloor_NullsAndMatch = data['floor_clean'].isnull() & MatchsFloor.notnull()
data.loc[maskFloor_NullsAndMatch, 'floor_clean'] = MatchsFloor[maskFloor_NullsAndMatch].apply(lambda x: float(0))

#Regex en la columna 'title'
MatchsFloor = data['title_lower'].apply(lambda x: x if x is np.NaN else regexFloor3.search(x))
maskFloor_NullsAndMatch = data['floor_clean'].isnull() & MatchsFloor.notnull()
data.loc[maskFloor_NullsAndMatch, 'floor_clean'] = MatchsFloor[maskFloor_NullsAndMatch].apply(lambda x: float(0))

In [ ]:
data['floor_clean'] = data['floor_clean'].astype(float)

In [ ]:
print("Columna 'floor'")
print("Cantidad de nulos antes del regex:", data.loc[maskApartment,'floor'].isnull().sum(), "("+str(round((data.loc[maskApartment,'floor'].isnull().sum()/data.loc[maskApartment,:].shape[0])*100,1))+"%)")
print("Cantidad de nulos después del regex:", data.loc[maskApartment,'floor_clean'].isnull().sum(),"("+str(round((data.loc[maskApartment,'floor_clean'].isnull().sum()/data.loc[maskApartment,:].shape[0])*100,1))+"%)")

Visualización

In [ ]:
maskNotNull_Floor = data['floor_clean'].notnull()
f, ax = plt.subplots()
f.set_figwidth(8)
f.set_figheight(7)
sns.boxplot(y=data.loc[maskNotNull_Floor,'floor_clean'])
plt.title("Distribución de pisos",fontsize=15)
plt.xlabel("Departamentos",fontsize=11)
plt.ylabel("Pisos",fontsize=11)

Análisis de outliers. Se consideran nulos a los pisos superiores a 20.

In [ ]:
maskOutliers = data['floor_clean'] > 20
maskOutliers.value_counts()

Se reemplaza los outliers por nulos

In [ ]:
maskOutliers = data['floor_clean'] > 20
data.loc[maskOutliers,'floor_clean'] = np.NaN

Visualización sin outliers

In [ ]:
f, ax = plt.subplots()
f.set_figwidth(8)
f.set_figheight(7)
sns.boxplot(y=data['floor_clean'])
plt.title("Distribución de pisos (sin outliers)",fontsize=15)
plt.xlabel("Departamentos",fontsize=11)
plt.ylabel("Pisos",fontsize=11)

---
<b><a id='section_2_6'>2.6. Columnas 'surface_total_in_m2' y 'surface_covered_in_m2'</a> </b>

In [ ]:
data['surface_total_in_m2_clean'] = data['surface_total_in_m2']

Análisis de nulos y valores

In [ ]:
data_describe.loc[data_describe.index.isin(['surface_covered_in_m2','surface_total_in_m2']),:]

In [ ]:
maskNull_coveredAndTotal = data['surface_covered_in_m2'].isnull() & data['surface_total_in_m2'].isnull()
print("Cantidad de nulos que coinciden:",maskNull_coveredAndTotal.value_counts()[True])

Se opta por completar la columna 'surface_total_in_m2' mediante el siguiente proceso:  
1- Imputar datos de acuerdo a la proporción media entre  'surface_covered_in_m2'  y 'surface_total_in_m2'  
2- Completamos los nulos restantes con datos extraidos de la columna 'descripcion' y 'title'

In [ ]:
maskNotNull_TotalAndCovered = data['surface_covered_in_m2'].notnull() & data['surface_total_in_m2'].notnull()
data['covered_pct'] = data.loc[maskNotNull_TotalAndCovered, 'surface_covered_in_m2'] / data.loc[maskNotNull_TotalAndCovered, 'surface_total_in_m2'] 
data['covered_pct'] = data['covered_pct'].apply(lambda x: 1 if x>1 else x)
meanByProperty = pd.DataFrame(data.groupby('property_type')['covered_pct'].mean())
meanByProperty.columns = ['covered_pct_mean']
print("Las casas son las que menos metros cuadrados tienen cubierto")
meanByProperty.sort_values(by = 'covered_pct_mean', ascending=False)

In [ ]:
data = data.merge(meanByProperty, on='property_type')

In [ ]:
maskFill = data['surface_covered_in_m2'].notnull() & data['surface_total_in_m2'].isnull()
surface_total_parche = data.loc[maskFill,'surface_covered_in_m2'] / data.loc[maskFill,'covered_pct_mean']
data.loc[maskFill, 'surface_total_in_m2_clean'] = surface_total_parche[maskFill] 

In [ ]:
print("Columna 'surface_total_in_m2'")
print("Cantidad de nulos antes de la imputación:", data['surface_total_in_m2'].isnull().sum(), "("+str(round((data['surface_total_in_m2'].isnull().sum()/data.shape[0])*100,1))+"%)")
print("Cantidad de nulos después de la imputación:", data['surface_total_in_m2_clean'].isnull().sum(),"("+str(round((data['surface_total_in_m2_clean'].isnull().sum()/data.shape[0])*100,1))+"%)")

In [ ]:
#Pattern
regexSurface = re.compile('(?P<mts2>\d\d?\d?\d?)\s?(?P<metros>mts|mt|metros|mt2|m2|m|mts2|m2c|mÂ²|mts²)')

#Regex en la columna 'descripcion'
MatchsSurface = data['description_lower'].apply(lambda x: x if x is np.NaN else regexSurface.search(x))
maskSurface_NullsAndMatch = data['surface_total_in_m2_clean'].isnull() & MatchsSurface.notnull()
data.loc[maskSurface_NullsAndMatch, 'surface_total_in_m2_clean'] = MatchsSurface[maskSurface_NullsAndMatch].apply(lambda x: float(x.group("mts2")))

#Regex en la columna 'title'
MatchsSurface = data['title_lower'].apply(lambda x: x if x is np.NaN else regexSurface.search(x))
maskSurface_NullsAndMatch = data['surface_total_in_m2_clean'].isnull() & MatchsSurface.notnull()
data.loc[maskSurface_NullsAndMatch, 'surface_total_in_m2_clean'] = MatchsSurface[maskSurface_NullsAndMatch].apply(lambda x: float(x.group("mts2")))

Se considera que una propiedad no puede tener menos de 20 mts cuadrados

In [ ]:
data['surface_total_in_m2_clean'] = data['surface_total_in_m2_clean'].apply(lambda x: np.NaN if x < 20 else x)

In [ ]:
print("Columna 'surface_total_in_m2'")
print("Cantidad de nulos antes del regex:", data['surface_total_in_m2'].isnull().sum(), "("+str(round((data['surface_total_in_m2'].isnull().sum()/data.shape[0])*100,1))+"%)")
print("Cantidad de nulos después del regex:", data['surface_total_in_m2_clean'].isnull().sum(),"("+str(round((data['surface_total_in_m2_clean'].isnull().sum()/data.shape[0])*100,1))+"%)")

Visualización

In [ ]:
maskNotNull_Surface = data['surface_total_in_m2_clean'].notnull()
f, ax = plt.subplots()
f.set_figwidth(12)
f.set_figheight(7)
sns.boxplot(x=data.loc[maskNotNull_Rooms,'property_type'],y=data.loc[maskNotNull_Rooms,'surface_total_in_m2_clean'])
plt.title("Cantidad de metros cuadrados según propiedad",fontsize=15)
plt.xlabel("Tipo de propiedad",fontsize=11)
plt.ylabel("Metros cuadrados",fontsize=11)

Análisis de outliers.  

In [ ]:
data['outliers_Surface'] = None
for i in data['property_type'].unique():
    rangoInterquartil_Surface= data.loc[data['property_type']==i,'surface_total_in_m2_clean'].quantile(0.75) - data.loc[data['property_type']==i,'surface_total_in_m2_clean'].quantile(0.25)
    umbral_lower_Surface = data.loc[data['property_type']==i,'surface_total_in_m2_clean'].quantile(0.25) - rangoInterquartil_Surface*1.5
    umbral_upper_Surface = data.loc[data['property_type']==i,'surface_total_in_m2_clean'].quantile(0.75) + rangoInterquartil_Surface*1.5


    data.loc[maskNotNull_Surface & (data['property_type']==i), 'outliers_Surface']= data.loc[maskNotNull_Surface & (data['property_type']==i), 'surface_total_in_m2_clean'].apply\
                                                                                    (lambda x: "Outlier superior" if x >umbral_upper_Surface else(
                                                                                 "Outlier inferior" if x<umbral_lower_Surface else 
                                                                                 "No es outlier"))

outliers_Surface = data['outliers_Surface']
data = data.drop(columns = ['outliers_Surface'])
pd.DataFrame(round((outliers_Surface.value_counts() / outliers_Surface[maskNotNull_Surface].shape[0])*100,1))

Se reemplaza a los outliers por nulos

In [ ]:
maskOutliers= outliers_Surface !="No es outlier"
data.loc[maskOutliers,'surface_total_in_m2_clean'] = np.NaN

Visualización sin outliers

In [ ]:
f, ax = plt.subplots()
f.set_figwidth(12)
f.set_figheight(7)

sns.boxplot(x=data['property_type'],y=data['surface_total_in_m2_clean'])
plt.title("Cantidad de metros cuadrados según propiedad (sin outliers)",fontsize=15)
plt.xlabel("Tipo de propiedad",fontsize=11)
plt.ylabel("Metros cuadrados",fontsize=11)

---
<b><a id='section_2_7'>2.7. Columna 'price_usd_per_m2'</a> </b>

Análisis de nulos y valores

In [ ]:
data_describe.loc[data_describe.index.isin(['price_usd_per_m2']),:]

Se reemplaza la columna 'price_usd_per_m2' por el cociente de las columnas 'price_aprox_usd_clean' y 'surface_total_in_m2_clean' para que sea consistente con los cambios realizados

In [ ]:
maskNotNull = data['price_aprox_usd_clean'].notnull() & data['surface_total_in_m2_clean'].notnull()
data['price_usd_per_m2_clean'] = (data.loc[maskNotNull, 'price_aprox_usd_clean'] / data.loc[maskNotNull, 'surface_total_in_m2_clean']).round(0)

Visualización

In [ ]:
f, ax = plt.subplots()
f.set_figwidth(12)
f.set_figheight(7)
sns.violinplot(data['property_type'], data['price_usd_per_m2_clean'])
plt.title("Precio por metro cuadrado",fontsize=15)
plt.xlabel("Tipo de propiedad",fontsize=11)
plt.ylabel("Precio por metro cuadrado",fontsize=11)

Análisis de outliers

In [ ]:
maskNotNull_pricexmt2 = data['price_usd_per_m2_clean'].notnull()
data['outliers_pricexmt2'] = None
for i in data['property_type'].unique():
    rangoInterquartil_pricexmt2= data.loc[data['property_type']==i,'price_usd_per_m2_clean'].quantile(0.75) - data.loc[data['property_type']==i,'price_usd_per_m2_clean'].quantile(0.25)
    umbral_lower_pricexmt2 = data.loc[data['property_type']==i,'price_usd_per_m2_clean'].quantile(0.25) - rangoInterquartil_pricexmt2*1.5
    umbral_upper_pricexmt2 = data.loc[data['property_type']==i,'price_usd_per_m2_clean'].quantile(0.75) + rangoInterquartil_pricexmt2*1.5


    data.loc[maskNotNull_pricexmt2 & (data['property_type']==i), 'outliers_pricexmt2']= data.loc[maskNotNull_pricexmt2 & (data['property_type']==i), 'price_usd_per_m2_clean'].apply\
                                                                                    (lambda x: "Outlier superior" if x >umbral_upper_pricexmt2 else(
                                                                                 "Outlier inferior" if x<umbral_lower_pricexmt2 else 
                                                                                 "No es outlier"))

outliers_pricexmt2 = data['outliers_pricexmt2']
data = data.drop(columns = ['outliers_pricexmt2'])
pd.DataFrame(round((outliers_pricexmt2.value_counts() / outliers_pricexmt2[maskNotNull_pricexmt2].shape[0])*100,1))

Se reemplaza a los outliers por nulos

In [ ]:
maskOutliers= outliers_pricexmt2 !="No es outlier"
data.loc[maskOutliers,'price_usd_per_m2_clean'] = np.NaN

Visualización sin outliers

In [ ]:
f, ax = plt.subplots()
f.set_figwidth(12)
f.set_figheight(7)
sns.violinplot(data['property_type'], data['price_usd_per_m2_clean'])
plt.title("Precio por metro cuadrado (sin outliers)",fontsize=15)
plt.xlabel("Tipo de propiedad",fontsize=11)
plt.ylabel("Precio por metro cuadrado",fontsize=11)

---
---

<b><a id='section_3'><h4>3. Nuevas variables</h4></a></b>
<a href='#section_3_1'>3.1. Pileta</a>  
<a href='#section_3_2'>3.2. Cochera  </a>  
<a href='#section_3_3'>3.3. Barrio cerrado</a>  
<a href='#section_3_4'>3.4. Jacuzzi  </a>  
<a href='#section_3_5'>3.5. Terraza  </a>  
<a href='#section_3_6'>3.6. Quincho  </a>  
<a href='#section_3_67'>3.7. Gimnasio  </a>  

Definimos una función para graficar los patrones encontrados

In [ ]:
data['decil'] = pd.qcut(data['price_aprox_usd_clean'],10,labels=[1,2,3,4,5,6,7,8,9,10])
def chart_type(y_value):
    chart_pattern = pd.DataFrame(data[y_value].value_counts())
    chart_pattern[y_value+" (%)"] = round(pd.DataFrame(data[y_value].value_counts())/data[y_value].shape[0]*100,2)
    
    chart_property = (data.groupby('property_type')[y_value].sum()/data.groupby('property_type')[y_value].count())*100
    chart_property = pd.DataFrame(round(chart_property,2))
    chart_property =chart_property.reset_index()
    
    chart_decil = (data.groupby('decil')[y_value].sum()/data.groupby('decil')[y_value].count())*100
    chart_decil = pd.DataFrame(round(chart_decil,2))
    chart_decil =chart_decil.reset_index()
    
    f, (ax1, ax2, ax3) = plt.subplots(1, 3)
    f.set_figwidth(12)
    f.set_figheight(4.5)
    
    ax1.pie(chart_pattern[y_value],labels=chart_pattern.index, autopct='%1.1f%%', shadow=True)  
    sns.barplot(x='property_type', y=y_value, data=chart_property, ax=ax2)
    sns.barplot(x='decil', y=y_value, data=chart_decil, color = "#009966", ax=ax3)
    
    plt.tight_layout()
    ax1.title.set_text("Patrones de \'"+y_value+"\' encontrados")
    ax2.title.set_text("Porcentaje de "+y_value+" por tipo de propiedad")
    ax3.title.set_text("Porcentaje de "+y_value+" según decil\ndel valor de la propiedad")
    return

---
<b><a id='section_3_1'>3.1. Pileta</a>  </b>

In [ ]:
#Pattern
regexPileta = re.compile("(?P<pileta>pileta|picina|piscina|pisina)")

data['pileta'] = 0
#Regex en la columna 'descripcion'
MatchsPileta = data['description_lower'].apply(lambda x: x if x is np.NaN else regexPileta.search(x))
maskPileta_Match = MatchsPileta.notnull()
data.loc[maskPileta_Match, 'pileta'] = 1

#Regex en la columna 'title'
MatchsPileta = data['title_lower'].apply(lambda x: x if x is np.NaN else regexPileta.search(x))
maskPileta_Match = MatchsPileta.notnull()
data.loc[maskPileta_Match, 'pileta'] = 1

chart_type('pileta')

---
<b><a id='section_3_2'>3.2. Cochera</a>  </b>

In [ ]:
#Pattern
regexCochera = re.compile("(?P<cochera>cochera|garage|auto\s|automovil|vehiculo|\scoche\s|\scoches\s)")

data['cochera'] = 0
#Regex en la columna 'descripcion'
MatchsCochera = data['description_lower'].apply(lambda x: x if x is np.NaN else regexCochera.search(x))
maskCochera_Match = MatchsCochera.notnull()
data.loc[maskCochera_Match, 'cochera'] = 1

#Regex en la columna 'title'
MatchsCochera = data['title_lower'].apply(lambda x: x if x is np.NaN else regexCochera.search(x))
maskCochera_Match = MatchsCochera.notnull()
data.loc[maskCochera_Match, 'cochera'] = 1

chart_type('cochera')

---
<b><a id='section_3_3'>3.3. Barrio cerrado</a>  </b>

In [ ]:
#Pattern
regexBarrioCerrado = re.compile("(?P<barrio>(barrio cerrado)|(barrio semicerrado)|(barrio semi cerrado)|country)")

data['BarrioCerrado'] = 0
#Regex en la columna 'descripcion'
MatchsBarrioCerrado = data['description_lower'].apply(lambda x: x if x is np.NaN else regexBarrioCerrado.search(x))
maskBarrioCerrado_Match = MatchsBarrioCerrado.notnull()
data.loc[maskBarrioCerrado_Match, 'BarrioCerrado'] = 1

#Regex en la columna 'title'
MatchsBarrioCerrado = data['title_lower'].apply(lambda x: x if x is np.NaN else regexBarrioCerrado.search(x))
maskBarrioCerrado_Match = MatchsBarrioCerrado.notnull()
data.loc[maskBarrioCerrado_Match, 'BarrioCerrado'] = 1

chart_type('BarrioCerrado')

---
<b><a id='section_3_4'>3.4. Jacuzzi</a>  </b>

In [ ]:
#Pattern
regexJacuzzi = re.compile("(?P<jacuzzi>(jacuzzi)|(suana)|(jacuzi))")

data['Jacuzzi'] = 0
#Regex en la columna 'descripcion'
MatchsJacuzzi = data['description_lower'].apply(lambda x: x if x is np.NaN else regexJacuzzi.search(x))
maskJacuzzi_Match = MatchsJacuzzi.notnull()
data.loc[maskJacuzzi_Match, 'Jacuzzi'] = 1

#Regex en la columna 'title'
MatchsJacuzzi = data['title_lower'].apply(lambda x: x if x is np.NaN else regexJacuzzi.search(x))
maskJacuzzi_Match = MatchsJacuzzi.notnull()
data.loc[maskJacuzzi_Match, 'Jacuzzi'] = 1

chart_type('Jacuzzi')

---
<b><a id='section_3_5'>3.5. Terraza</a>  </b>

In [ ]:
#Pattern
regexTerraza = re.compile("(?P<terraza>(terraza)|(terrasa)|(azotea)|(asotea))")

data['Terraza'] = 0
#Regex en la columna 'descripcion'
MatchsTerraza = data['description_lower'].apply(lambda x: x if x is np.NaN else regexTerraza.search(x))
maskTerraza_Match = MatchsTerraza.notnull()
data.loc[maskTerraza_Match, 'Terraza'] = 1

#Regex en la columna 'title'
MatchsTerraza = data['title_lower'].apply(lambda x: x if x is np.NaN else regexTerraza.search(x))
maskTerraza_Match = MatchsTerraza.notnull()
data.loc[maskTerraza_Match, 'Terraza'] = 1

chart_type('Terraza')

---
<b><a id='section_3_6'>3.6. Quincho</a>  </b>

In [ ]:
#Pattern
regexQuincho = re.compile("(?P<quincho>(quincho)|(kincho)|(qincho)|(\ssum\s)|(s\.u\.m\.))")

data['Quincho'] = 0
#Regex en la columna 'descripcion'
MatchsQuincho = data['description_lower'].apply(lambda x: x if x is np.NaN else regexQuincho.search(x))
maskQuincho_Match = MatchsQuincho.notnull()
data.loc[maskQuincho_Match, 'Quincho'] = 1

#Regex en la columna 'title'
MatchsQuincho = data['title_lower'].apply(lambda x: x if x is np.NaN else regexQuincho.search(x))
maskQuincho_Match = MatchsQuincho.notnull()
data.loc[maskQuincho_Match, 'Quincho'] = 1

chart_type('Quincho')

---
<b><a id='section_3_7'>3.7. Gimnasio</a>  </b>

In [ ]:
#Pattern
regexGimnasio = re.compile("(?P<gym>(gym)|(gimnasio)|(gimnacio))")

data['Gimnasio'] = 0
#Regex en la columna 'descripcion'
MatchsGimnasio = data['description_lower'].apply(lambda x: x if x is np.NaN else regexGimnasio.search(x))
maskGimnasio_Match = MatchsGimnasio.notnull()
data.loc[maskGimnasio_Match, 'Gimnasio'] = 1

#Regex en la columna 'title'
MatchsGimnasio = data['title_lower'].apply(lambda x: x if x is np.NaN else regexGimnasio.search(x))
maskGimnasio_Match = MatchsGimnasio.notnull()
data.loc[maskGimnasio_Match, 'Gimnasio'] = 1

chart_type('Gimnasio')

---
<b><a id='section_3_9'>3.9. Seguridad</a>  </b>

In [ ]:
#Pattern
regexSeguridad = re.compile("(?P<seguridad>seguridad|vigilancia|c[a|á]maras|ojo de halc[o|ó]n|vigilador|garita|circuito cerrado|cctv|monitoreo|seg 24|24hr)+")

data['Seguridad'] = 0
#Regex en la columna 'descripcion'
MatchsSeguridad = data['description_lower'].apply(lambda x: x if x is np.NaN else regexSeguridad.search(x))
maskSeguridad_Match = MatchsSeguridad.notnull()
data.loc[maskSeguridad_Match, 'Seguridad'] = 1

#Regex en la columna 'title'
MatchsSeguridad = data['title_lower'].apply(lambda x: x if x is np.NaN else regexSeguridad.search(x))
maskSeguridad_Match = MatchsSeguridad.notnull()
data.loc[maskSeguridad_Match, 'Seguridad'] = 1

chart_type('Seguridad')

---
<b><a id='section_3_10'>3.10. A Reciclar</a>  </b>

In [ ]:
#Pattern
regexReciclar = re.compile("(?P<reciclar>reciclar|refaccionar|acondicionar|a terminar|para terminar)+")

data['Reciclar'] = 0
#Regex en la columna 'descripcion'
MatchsReciclar = data['description_lower'].apply(lambda x: x if x is np.NaN else regexReciclar.search(x))
maskReciclar_Match = MatchsReciclar.notnull()
data.loc[maskReciclar_Match, 'Reciclar'] = 1

#Regex en la columna 'title'
MatchsReciclar = data['title_lower'].apply(lambda x: x if x is np.NaN else regexReciclar.search(x))
maskReciclar_Match = MatchsReciclar.notnull()
data.loc[maskReciclar_Match, 'Reciclar'] = 1

chart_type('Reciclar')

---
<b><a id='section_3_11'>3.11. De Epoca</a>  </b>

In [ ]:
#Pattern
regexEpoca = re.compile("(?P<epoca>de [é|e]poca)+")

data['Epoca'] = 0
#Regex en la columna 'descripcion'
MatchsEpoca = data['description_lower'].apply(lambda x: x if x is np.NaN else regexEpoca.search(x))
maskEpoca_Match = MatchsEpoca.notnull()
data.loc[maskEpoca_Match, 'Epoca'] = 1

#Regex en la columna 'title'
MatchsEpoca = data['title_lower'].apply(lambda x: x if x is np.NaN else regexEpoca.search(x))
maskEpoca_Match = MatchsEpoca.notnull()
data.loc[maskEpoca_Match, 'Epoca'] = 1

chart_type('Epoca')

---
<b><a id=''> Cantidad de Cocheras</a>  </b>

In [ ]:
#Primer pattern
regexCantCoch = re.compile('(?P<numero>\d{1})\s?(cochera|garage|auto\s|automovil|vehiculo|\scoche\s|\scoches\s)')

data['cantcocheras'] = np.nan

#Regex en la columna 'descripcion'
MatchsCantCoch = data['description_lower'].apply(lambda x: x if x is np.NaN else regexCantCoch.search(x))
maskCantCoch_NullsAndMatch = data['cantcocheras'].isnull() & MatchsCantCoch.notnull()
data.loc[maskCantCoch_NullsAndMatch, 'cantcocheras'] = MatchsCantCoch[maskCantCoch_NullsAndMatch].apply(lambda x: float(x.group("numero")))

#Regex en la columna 'title'
MatchsCantCoch = data['title_lower'].apply(lambda x: x if x is np.NaN else regexCantCoch.search(x))
maskCantCoch_NullsAndMatch = data['cantcocheras'].isnull() & MatchsCantCoch.notnull()
data.loc[maskCantCoch_NullsAndMatch, 'cantcocheras'] = MatchsCantCoch[maskCantCoch_NullsAndMatch].apply(lambda x: float(x.group("numero")))

data['cantcocheras'].value_counts()


In [ ]:
#Segundo pattern
regexCantCoch2 = re.compile('(?P<numeroPalabra>uno|un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve)\s?(cochera|garage|auto\s|automovil|vehiculo|\scoche\s|\scoches\s)')

#Regex en la columna 'descripcion' usando la función MatchtoNumber
MatchsCantCoch2 = data['description_lower'].apply(lambda x: x if x is np.NaN else regexCantCoch2.search(x))
maskCantCoch2_NullsAndMatch = data['cantcocheras'].isnull() & MatchsCantCoch2.notnull()
data.loc[maskCantCoch2_NullsAndMatch, 'cantcocheras'] = MatchsCantCoch2[maskCantCoch2_NullsAndMatch].apply(MatchToNumber)

#Regex en la columna 'title' usando la función MatchtoNumber
MatchsCantCoch2 = data['title_lower'].apply(lambda x: x if x is np.NaN else regexAmbientes2.search(x))
maskCantCoch2_NullsAndMatch = data['cantcocheras'].isnull() & MatchsCantCoch2.notnull()
data.loc[maskCantCoch2_NullsAndMatch, 'cantcocheras'] = MatchsCantCoch2[maskCantCoch2_NullsAndMatch].apply(MatchToNumber)

data['cantcocheras'].value_counts()

---
<b><a id=''> Cantidad de baños</a>  </b>

In [ ]:
# Primer Pattern
regexBanos = re.compile("(?P<numero>\d{1})\s?(baño|baños|banos|bano|banio|banios|baÃ±os|aÃ±os|toilette|toilettes|baÃ‘o)")

data['banos'] = np.nan

#Regex en la columna 'descripcion'
MatchsBanos = data['description_lower'].apply(lambda x: x if x is np.NaN else regexBanos.search(x))
maskBanos_NullsAndMatch = data['banos'].isnull() & MatchsBanos.notnull()
data.loc[maskBanos_NullsAndMatch, 'banos'] = MatchsBanos[maskBanos_NullsAndMatch].apply(lambda x: float(x.group("numero")))

#Regex en la columna 'title'
MatchsBanos = data['title_lower'].apply(lambda x: x if x is np.NaN else regexBanos.search(x))
maskBanos_NullsAndMatch = data['banos'].isnull() & MatchsBanos.notnull()
data.loc[maskBanos_NullsAndMatch, 'banos'] = MatchsBanos[maskBanos_NullsAndMatch].apply(lambda x: float(x.group("numero")))

data['banos'].value_counts()


In [ ]:
# Segundo Pattern
regexBanos = re.compile("(?P<numeroPalabra>uno|un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve)\s?(baño|baños|banos|bano|banio|banios|baÃ±os|aÃ±os|toilette|toilettes|baÃ‘o)")

#Regex en la columna 'descripcion'
MatchsBanos = data['description_lower'].apply(lambda x: x if x is np.NaN else regexBanos.search(x))
maskBanos_NullsAndMatch = data['banos'].isnull() & MatchsBanos.notnull()
data.loc[maskBanos_NullsAndMatch, 'banos'] = MatchsBanos[maskBanos_NullsAndMatch].apply(MatchToNumber)

#Regex en la columna 'title'
MatchsBanos = data['title_lower'].apply(lambda x: x if x is np.NaN else regexBanos.search(x))
maskBanos_NullsAndMatch = data['banos'].isnull() & MatchsBanos.notnull()
data.loc[maskBanos_NullsAndMatch, 'banos'] = MatchsBanos[maskBanos_NullsAndMatch].apply(MatchToNumber)

data['banos'].value_counts()

In [ ]:
data['banos'] = data.banos.fillna(1)
data.banos.value_counts()

---
<b><a id=''> Balcón</a>  </b>

In [ ]:
#Pattern
regexBalcon = re.compile("(?P<balcon>balcon|balcón|balcones)")

data['balcon'] = 0
#Regex en la columna 'descripcion'
MatchsBalcon = data['description_lower'].apply(lambda x: x if x is np.NaN else regexBalcon.search(x))
maskBalcon_Match = MatchsBalcon.notnull()
data.loc[maskBalcon_Match, 'balcon'] = 1

#Regex en la columna 'title'
MatchsBalcon = data['title_lower'].apply(lambda x: x if x is np.NaN else regexBalcon.search(x))
maskBalcon_Match = MatchsBalcon.notnull()
data.loc[maskBalcon_Match, 'balcon'] = 1

chart_type('balcon')

---
<b><a id=''> A estrenar</a>  </b>

In [ ]:
#Pattern
regexEstrenar = re.compile("(?P<Estrenar>(a estrenar)|(departamento nuevo))")

data['estrenar'] = 0
#Regex en la columna 'descripcion'
MatchsEstrenar = data['description_lower'].apply(lambda x: x if x is np.NaN else regexEstrenar.search(x))
maskEstrenar_Match = MatchsEstrenar.notnull()
data.loc[maskEstrenar_Match, 'estrenar'] = 1

#Regex en la columna 'title'
MatchsEstrenar = data['title_lower'].apply(lambda x: x if x is np.NaN else regexEstrenar.search(x))
maskEstrenar_Match = MatchsEstrenar.notnull()
data.loc[maskEstrenar_Match, 'estrenar'] = 1

chart_type('estrenar')

---
<b><a id=''> Calefacción central / Losa radiante</a>  </b>

In [ ]:
#Pattern
regexCalefaccion = re.compile("(?P<Calefaccion>(calefacciÃ³n por radiador)|(calefacciÃ³n central)|(calefaccion central)| \
(calefacción central)|(calefacciÃ³n por radiadores)|(calefaccion por losa radiante)|(calefacción por losa radiante))")

data['calefaccion'] = 0
#Regex en la columna 'descripcion'
MatchsCalefaccion = data['description_lower'].apply(lambda x: x if x is np.NaN else regexCalefaccion.search(x))
maskCalefaccion_Match = MatchsCalefaccion.notnull()
data.loc[maskCalefaccion_Match, 'calefaccion'] = 1

#Regex en la columna 'title'
MatchsCalefaccion = data['title_lower'].apply(lambda x: x if x is np.NaN else regexCalefaccion.search(x))
maskCalefaccion_Match = MatchsCalefaccion.notnull()
data.loc[maskCalefaccion_Match, 'calefaccion'] = 1

chart_type('calefaccion')

---
<b><a id='section_3_12'>3.12. Parque, Patio, Jardin</a>  </b>

In [ ]:
#Pattern
regexJardin = re.compile("(?P<parque>con parque|al parque|jard[i|í]n|patio)+")

data['Jardin'] = 0
#Regex en la columna 'descripcion'
MatchsJardin = data['description_lower'].apply(lambda x: x if x is np.NaN else regexJardin.search(x))
maskJardin_Match = MatchsJardin.notnull()
data.loc[maskJardin_Match, 'Jardin'] = 1

#Regex en la columna 'title'
MatchsJardin = data['title_lower'].apply(lambda x: x if x is np.NaN else regexJardin.search(x))
maskJardin_Match = MatchsJardin.notnull()
data.loc[maskJardin_Match, 'Jardin'] = 1

chart_type('Jardin')

---
---
<b><a id='section_4'><h4>4. Dataset final</h4></a></b>

Se construye un dataset con las variables que serán utilizadas en el modelo

In [ ]:
data_final = data[['property_type','provincia','ciudad','departamento','barrio',
                   'price_aprox_usd_clean','surface_total_in_m2_clean','price_usd_per_m2_clean','rooms_clean',
                  'pileta','cochera','BarrioCerrado', 'Jacuzzi','Terraza','Quincho','Gimnasio', 'Seguridad', 'Reciclar', 'cantcocheras', 'banos', 'balcon', 'estrenar', 'calefaccion', 'Jardin',]]

Se conserva únicamente las observaciones donde el <u>precio por metro cuadrado no es nulo</u>, ya que esta será la variable a estimar en el modelo

In [ ]:
mask = data['price_usd_per_m2_clean'].notnull()
data_final = data_final.loc[mask,:]

In [ ]:
data_describe_final = pd.DataFrame(round(data_final.isnull().sum() /data_final.shape[0],2), columns= ['Porcentaje de nulos'])
data_describe_final['type'] = data_final.dtypes
data_describe_final = data_describe_final[['type','Porcentaje de nulos']]
values= []
example=[]
for x in data_final:
    values.append(len(data_final[x].unique()))
    example.append(data_final[x].unique())
data_describe_final['Valores distintos'] = values
data_describe_final['Ejemplos'] = example

print("El dataset que se utilizará para realizar el modelo tiene",data_final.shape[0]," observaciones.")
data_describe_final

In [ ]:
data_final.to_csv("../data")


In [ ]:
data_final = pd.get_dummies(data = data_final, columns = ["ciudad", "property_type"], drop_first = True)



In [ ]:
data_final.columns

data_final_model = data_final

In [ ]:
data_final = data_final_model
data_final = data_final.drop(["barrio", "departamento", 'cantcocheras'], axis = 1)

data_final.dropna(inplace = True)
data_final["constant"] = 1

In [ ]:
data_final.shape

In [ ]:
data_features = data_final.drop(["price_aprox_usd_clean","price_usd_per_m2_clean","provincia"\
                                ], axis = 1)

target = data_final["price_usd_per_m2_clean"]



In [ ]:
import statsmodels.api as sm

from sklearn import linear_model
from sklearn.model_selection import train_test_split



Xtrain , Xtest , ytrain , ytest = train_test_split(data_features,target)

Xtrain = sm.add_constant(Xtrain)
model_sm = sm.OLS(ytrain,Xtrain).fit()

In [ ]:
model_sm.summary()

In [ ]:
#from sm import cvxopt
model_reg_lasso = sm.OLS(ytrain,Xtrain).fit_regularized(method='elastic_net', alpha=0.1, L1_wt = 1)

In [ ]:
#model_reg_lasso.score

In [ ]:
prediction_sm = model_sm.predict(Xtest)
prediction_lasso = model_reg_lasso.predict(Xtest)



from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print ('R2 Statmodels OLS:', r2_score(ytest, prediction_sm).round(2))
print ('R2 Statmodels Lasso:', r2_score(ytest, prediction_lasso).round(2))





In [ ]:
model_skl_LassoCV = linear_model.LassoCV(alphas=np.linspace(0.001,1, 1000), cv=3, normalize=True)
model_skl_LassoCV.fit(Xtrain, ytrain)
predictions_lassoCV_skl = model_skl_LassoCV.predict(Xtest)



In [ ]:

print ("R2 SkLearn LassoCV:", model_skl_LassoCV.score(Xtest, ytest))
print ("R2 SkLearn LassoCV:", r2_score(ytest, predictions_lassoCV_skl))
print ("alpha:", model_skl_LassoCV.alpha_)

#print ("Coef:", model_skl_LassoCV.coef_)
#print ((model_skl_LassoCV.coef_).shape)
#print((data_features.columns).shape)

betas = pd.DataFrame(data= model_skl_LassoCV.coef_ , index=data_features.columns, columns=['Betas'])
pd.set_option('display.max_rows', 600)
#betas


In [ ]:
betas_0_mask = betas.Betas == 0
betas.loc[betas_0_mask,:]

In [ ]:
data_final.columns

In [ ]:
dataCiudad = data_final


In [ ]:
### regresion para capital federal

In [359]:
dataCiudad = data[['property_type','provincia','ciudad','departamento','barrio',
                   'price_aprox_usd_clean','surface_total_in_m2_clean','price_usd_per_m2_clean','rooms_clean',
                  'pileta','cochera','BarrioCerrado', 'Jacuzzi','Terraza','Quincho','Gimnasio', 'Seguridad', 'Reciclar', 'cantcocheras', 'banos', 'balcon', 'estrenar', 'calefaccion', 'Jardin',]]

In [360]:
dataCiudad = dataCiudad[dataCiudad.provincia == "Capital Federal"]
dataCiudad = dataCiudad.drop(["barrio", "departamento", 'cantcocheras'], axis = 1)

dataCiudad.head()

,property_type,provincia,ciudad,price_aprox_usd_clean,surface_total_in_m2_clean,price_usd_per_m2_clean,rooms_clean,pileta,cochera,BarrioCerrado,...,Terraza,Quincho,Gimnasio,Seguridad,Reciclar,banos,balcon,estrenar,calefaccion,Jardin
118,house,Capital Federal,Belgrano,NaN,464.000000,NaN,5.0,0,1,0,...,1,0,0,1,0,1.0,1,0,0,1
168,house,Capital Federal,Flores,265000.0,228.552206,1159.0,4.0,1,1,0,...,1,1,0,0,0,1.0,0,0,0,0
171,house,Capital Federal,Caballito,NaN,395.000000,NaN,4.0,1,1,0,...,1,0,0,1,0,1.0,1,0,0,0
176,house,Capital Federal,Nuñez,820000.0,420.000000,1952.0,2.0,0,1,0,...,1,1,0,1,0,2.0,1,0,0,1
177,house,Capital Federal,Belgrano,820000.0,420.000000,1952.0,2.0,0,1,0,...,1,1,0,1,0,2.0,1,0,0,1


In [361]:
dataCiudad = pd.get_dummies(data = dataCiudad, columns = ["ciudad", "property_type"], drop_first = True)
dataCiudad.head()

,provincia,price_aprox_usd_clean,surface_total_in_m2_clean,price_usd_per_m2_clean,rooms_clean,pileta,cochera,BarrioCerrado,Jacuzzi,Terraza,...,ciudad_Villa Pueyrredón,ciudad_Villa Real,ciudad_Villa Riachuelo,ciudad_Villa Santa Rita,ciudad_Villa Soldati,ciudad_Villa Urquiza,ciudad_Villa del Parque,property_type_apartment,property_type_house,property_type_store
118,Capital Federal,NaN,464.000000,NaN,5.0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
168,Capital Federal,265000.0,228.552206,1159.0,4.0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
171,Capital Federal,NaN,395.000000,NaN,4.0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
176,Capital Federal,820000.0,420.000000,1952.0,2.0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
177,Capital Federal,820000.0,420.000000,1952.0,2.0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [362]:
dataCiudad.dropna(inplace = True)
dataCiudadTarget = dataCiudad.price_usd_per_m2_clean

dataCiudadFeatures = dataCiudad.drop(['price_aprox_usd_clean','price_usd_per_m2_clean','provincia'\
                             ], axis = 1)


dataCiudadFeatures["constant"] = 1

In [336]:

dataCiudadFeatures.head()

,surface_total_in_m2_clean,rooms_clean,pileta,cochera,BarrioCerrado,Jacuzzi,Terraza,Quincho,Gimnasio,Seguridad,...,ciudad_Villa Real,ciudad_Villa Riachuelo,ciudad_Villa Santa Rita,ciudad_Villa Soldati,ciudad_Villa Urquiza,ciudad_Villa del Parque,property_type_apartment,property_type_house,property_type_store,constant
168,228.552206,4.0,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,1
176,420.000000,2.0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,1,0,1
177,420.000000,2.0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,1,0,1
253,637.921032,3.0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
261,271.000000,3.0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [260]:
#dataCiudad1Features = dataCiudad1Features.reset_index()
#dataCiudad1Features.reset_index
#dataCiudad1Features.drop(columns = ["index"])


In [318]:
dataCiudad1Features.head()

,surface_total_in_m2_clean,rooms_clean,pileta,cochera,BarrioCerrado,Jacuzzi,Terraza,Quincho,Gimnasio,Seguridad,...,ciudad_Villa Pueyrredón,ciudad_Villa Real,ciudad_Villa Riachuelo,ciudad_Villa Santa Rita,ciudad_Villa Soldati,ciudad_Villa Urquiza,ciudad_Villa del Parque,property_type_apartment,property_type_house,property_type_store
168,228.552206,4.0,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
176,420.000000,2.0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,1,0
177,420.000000,2.0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,1,0
253,637.921032,3.0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
261,271.000000,3.0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [363]:
Xtrain , Xtest , ytrain , ytest = train_test_split(dataCiudad1Features,dataCiudad1Targets)

#Xtrain = sm.add_constant(Xtrain)
model_sm = sm.OLS(ytrain,Xtrain).fit()

model_reg_lasso = sm.OLS(ytrain,Xtrain).fit_regularized(method='elastic_net', alpha=0.02, L1_wt = 1)

In [309]:
#model_sm.summary()
print(Xtest.shape)
print(ytest.shape)
type(ytest)

(4373, 75)
(4373,)


pandas.core.series.Series

In [364]:
prediction_sm = model_sm.predict(Xtest)




#from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print ('R2 Statmodels OLS:', r2_score(ytest, prediction_sm).round(2))



R2 Statmodels OLS: 0.57


In [ ]:
####regresion para NO capital federal

In [341]:
dataNoCiudad = data[['property_type','provincia','ciudad','departamento','barrio',
                   'price_aprox_usd_clean','surface_total_in_m2_clean','price_usd_per_m2_clean','rooms_clean',
                  'pileta','cochera','BarrioCerrado', 'Jacuzzi','Terraza','Quincho','Gimnasio', 'Seguridad', 'Reciclar', 'cantcocheras', 'banos', 'balcon', 'estrenar', 'calefaccion', 'Jardin',]]

In [344]:
dataNoCiudad = dataNoCiudad[dataNoCiudad.provincia != "Capital Federal"]
dataNoCiudad = dataNoCiudad.drop(["barrio", "departamento", 'cantcocheras'], axis = 1)

dataNoCiudad.head()

,property_type,provincia,ciudad,price_aprox_usd_clean,surface_total_in_m2_clean,price_usd_per_m2_clean,rooms_clean,pileta,cochera,BarrioCerrado,...,Terraza,Quincho,Gimnasio,Seguridad,Reciclar,banos,balcon,estrenar,calefaccion,Jardin
0,house,Bs.As. G.B.A. Zona Norte,Tigre,NaN,316.000000,NaN,3.0,1,1,0,...,1,0,0,1,0,1.0,1,0,0,0
1,house,Bs.As. G.B.A. Zona Norte,Tigre,345000.0,NaN,NaN,3.0,1,1,1,...,1,0,0,1,0,1.0,0,0,0,0
2,house,Bs.As. G.B.A. Zona Norte,Tigre,NaN,303.000000,NaN,3.0,1,1,0,...,1,0,0,1,0,1.0,0,0,0,1
3,house,Bs.As. G.B.A. Zona Norte,Tigre,NaN,335.000000,NaN,3.0,1,0,0,...,1,0,0,1,0,1.0,0,0,0,1
4,house,Bs.As. G.B.A. Zona Norte,Tigre,437000.0,296.539255,1474.0,3.0,1,0,0,...,1,0,0,1,0,1.0,0,0,0,0


In [345]:
dataNoCiudad = pd.get_dummies(data = dataNoCiudad, columns = ["ciudad", "property_type"], drop_first = True)
dataNoCiudad.head()

,provincia,price_aprox_usd_clean,surface_total_in_m2_clean,price_usd_per_m2_clean,rooms_clean,pileta,cochera,BarrioCerrado,Jacuzzi,Terraza,...,ciudad_Villa del Totoral,ciudad_Villarino,ciudad_Vistalba,ciudad_Wenceslao Escalante,ciudad_Yerba Buena,ciudad_Zárate,ciudad_Álvarez,property_type_apartment,property_type_house,property_type_store
0,Bs.As. G.B.A. Zona Norte,NaN,316.000000,NaN,3.0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,Bs.As. G.B.A. Zona Norte,345000.0,NaN,NaN,3.0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,1,0
2,Bs.As. G.B.A. Zona Norte,NaN,303.000000,NaN,3.0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,Bs.As. G.B.A. Zona Norte,NaN,335.000000,NaN,3.0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,Bs.As. G.B.A. Zona Norte,437000.0,296.539255,1474.0,3.0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [348]:
dataNoCiudad.dropna(inplace = True)
dataNoCiudadTarget = dataNoCiudad.price_usd_per_m2_clean

dataNoCiudadFeatures = dataNoCiudad.drop(['price_aprox_usd_clean','price_usd_per_m2_clean','provincia'\
                             ], axis = 1)


dataNoCiudadFeatures["constant"] = 1

In [349]:
dataNoCiudadFeatures.head()

,surface_total_in_m2_clean,rooms_clean,pileta,cochera,BarrioCerrado,Jacuzzi,Terraza,Quincho,Gimnasio,Seguridad,...,ciudad_Villarino,ciudad_Vistalba,ciudad_Wenceslao Escalante,ciudad_Yerba Buena,ciudad_Zárate,ciudad_Álvarez,property_type_apartment,property_type_house,property_type_store,constant
4,296.539255,3.0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,1
5,318.237249,3.0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
6,303.771920,3.0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
7,700.000000,3.0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
15,180.000000,3.0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1


In [355]:
XtrainN , XtestN , ytrainN , ytestN = train_test_split(dataNoCiudadFeatures,dataNoCiudadTarget)

#Xtrain = sm.add_constant(Xtrain)
model_smN = sm.OLS(ytrainN,XtrainN).fit()

#model_reg_lasso = sm.OLS(ytrain,Xtrain).fit_regularized(method='elastic_net', alpha=0.02, L1_wt = 1)

In [366]:
model_smN.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     price_usd_per_m2_clean   R-squared:                       0.603
Model:                                OLS   Adj. R-squared:                  0.599
Method:                     Least Squares   F-statistic:                     161.1
Date:                    Sun, 21 Jun 2020   Prob (F-statistic):               0.00
Time:                            16:42:02   Log-Likelihood:            -2.4707e+05
No. Observations:                   32841   AIC:                         4.948e+05
Df Residuals:                       32533   BIC:                         4.973e+05
Df Model:                             307                                         
Covariance Type:                nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
surface_total_in_m2_clean                     -1.8597      0.024    -77.533      0.000      -1.907      -1.813
rooms_clean                                  -13.4762      3.397     -3.967      0.000     -20.134      -6.818
pileta                                       183.3324      7.008     26.159      0.000     169.596     197.069
cochera                                       57.7813      5.340     10.820      0.000      47.314      68.248
BarrioCerrado                                 -6.7878     14.558     -0.466      0.641     -35.323      21.747
Jacuzzi                                      227.5981     17.398     13.081      0.000     193.496     261.700
Terraza                                      -13.6671      6.293     -2.172      0.030     -26.001      -1.333
Quincho                                       16.3812      7.638      2.145      0.032       1.411      31.352
Gimnasio                                      96.2840     12.262      7.852      0.000      72.250     120.318
Seguridad                                      1.4416      9.254      0.156      0.876     -16.696      19.579
Reciclar                                    -210.9246     15.280    -13.804      0.000    -240.874    -180.975
banos                                         34.3619      6.690      5.136      0.000      21.249      47.474
balcon                                        71.6882      6.085     11.780      0.000      59.760      83.616
estrenar                                      73.8829      8.001      9.234      0.000      58.200      89.566
calefaccion                                  155.4625     12.726     12.216      0.000     130.520     180.405
Jardin                                      -103.0585      5.796    -17.782      0.000    -114.418     -91.698
ciudad_Adolfo Alsina                        3.183e-10   2.66e-10      1.196      0.232   -2.03e-10     8.4e-10
ciudad_Agua Blanca                          -260.2885    450.121     -0.578      0.563   -1142.542     621.965
ciudad_Agua de Oro                          -293.9689    450.473     -0.653      0.514   -1176.912     588.974
ciudad_Aguas Verdes                         -224.1019    318.497     -0.704      0.482    -848.367     400.163
ciudad_Aldea Brasilera                      -109.6454    450.191     -0.244      0.808    -992.036     772.745
ciudad_Aldea Salto                          4.915e-10   3.83e-10      1.283      0.200   -2.59e-10    1.24e-09
ciudad_Alejandro Roca                        3.08e-10   3.06e-10      1.006      0.314   -2.92e-10    9.08e-10
ciudad_Almafuerte                          -9.689e-11   5.42e-10     -0.179      0.858   -1.16e-09    9.66e-10
ciudad_Almirante Brown                    